# 0. INFO-GAN

### Reference
- https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/infogan/infogan.py
- paper : https://arxiv.org/abs/1606.03657

# 1. Library Import

In [1]:
!pip install easydict

  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6350 sha256=cbff704c6c73ef92ed008b4189ea7bb9ba35e78646101df9b12cbf14635b3f72
  Stored in directory: /root/.cache/pip/wheels/88/96/68/c2be18e7406804be2e593e1c37845f2dd20ac2ce1381ce40b0
Successfully built easydict


In [2]:
import easydict
import os
import numpy as np
import math
import itertools

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [3]:
os.makedirs("images/static/", exist_ok = True)
os.makedirs("images/varying_c1/", exist_ok = True)
os.makedirs("images/varying_c2/", exist_ok = True)

# 2. Parameter Setting

In [4]:
opt = easydict.EasyDict({"n_epochs" : 5, "batch_size" : 64,
                         "lr" : 0.0002, "b1" : 0.5, "b2" : 0.999,
                         "n_cpu" : 8, "latent_dim" : 62,
                         "code_dim" : 2, "n_classes" : 10,
                         "img_size" : 32, "channels" : 1,
                         "sample_interval" : 400})

In [5]:
print(opt)

{'n_epochs': 5, 'batch_size': 64, 'lr': 0.0002, 'b1': 0.5, 'b2': 0.999, 'n_cpu': 8, 'latent_dim': 62, 'code_dim': 2, 'n_classes': 10, 'img_size': 32, 'channels': 1, 'sample_interval': 400}


In [6]:
cuda = True if torch.cuda.is_available() else False

In [7]:
print(cuda)

True


# 3. Model

## 3.1 Weights Init

In [8]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

## 3.2 One-hot Encoder

In [9]:
def to_categorical(y, num_columns):
    """
        Return one-hot encoded Variable
    """
    y_cat = np.zeros((y.shape[0], num_columns))
    y_cat[range(y.shape[0]), y] = 1.0
    
    return Variable(FloatTensor(y_cat))

## 3.3 Generator

In [10]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # code dimension ?????
        input_dim = opt.latent_dim + opt.n_classes + opt.code_dim
        
        # Initial size before upsampling
        # 2 ** #upsampling
        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(input_dim, 128 * self.init_size ** 2))
        
        self.conv_blocks = nn.Sequential(nn.BatchNorm2d(128),
                                         nn.Upsample(scale_factor = 2),
                                         nn.Conv2d(128, 128, 3, stride = 1, padding = 1),
                                         nn.BatchNorm2d(128, 0.8),
                                         nn.LeakyReLU(0.2, inplace = True),
                                         nn.Upsample(scale_factor = 2),
                                         nn.Conv2d(128, 64, 3, stride = 1, padding = 1),
                                         nn.BatchNorm2d(64, 0.8),
                                         nn.LeakyReLU(0.2, inplace = True),
                                         nn.Conv2d(64, opt.channels, 3, stride = 1, padding = 1),
                                         nn.Tanh()
                                        )
        
    def forward(self, noise, labels, code):
        # Concatenates
        gen_input = torch.cat((noise, labels, code), -1)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

## 3.4 Discriminator

In [11]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        def discriminator_block(in_filters, out_filters, bn = True):
            """
                Returns layers of each discriminator block
            """
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1),
                     nn.LeakyReLU(0.2, inplace = True),
                     nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block
        
        self.conv_blocks = nn.Sequential(*discriminator_block(opt.channels, 16, bn = False),
                                         *discriminator_block(16, 32),
                                         *discriminator_block(32, 64),
                                         *discriminator_block(64, 128)
                                        )
        
        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4
        
        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1))
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, opt.n_classes), nn.Softmax())
        self.latent_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, opt.code_dim))
        
    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)
        latent_code = self.latent_layer(out)
        
        return validity, label, latent_code

## 3.5 Loss Functions

In [12]:
adversarial_loss = torch.nn.MSELoss()
categorical_loss = torch.nn.CrossEntropyLoss()
continuous_loss = torch.nn.MSELoss()

In [13]:
# Loss Weights
lambda_cat = 1
lambda_con = 0.1

# 4. Data Loader and Model Training

## 4.1 Model Initialize

In [14]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [15]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    categorical_loss.cuda()
    continuous_loss.cuda()

In [16]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (conv_blocks): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ad

## 4.2 Data Loader

In [17]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2021-04-26 14:16:12--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-04-26 14:16:12--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [               <=>  ]  33.20M  10.4MB/s    in 3.2s    

2021-04-26 14:16:16 (10.4 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [18]:
# Condigure data loader
os.makedirs("./", exist_ok = True)
dataloader = torch.utils.data.DataLoader(
                datasets.MNIST(
                    "./",
                    train = True,
                    download = True,
                    transform = transforms.Compose(
                        [transforms.Resize(opt.img_size), transforms.ToTensor(),
                         transforms.Normalize([0.5], [0.5])]
                    )
                ),
                batch_size = opt.batch_size,
                shuffle = True
            )

## 4.3 Optimizers

In [19]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))
optimizer_info = torch.optim.Adam(itertools.chain(generator.parameters(),
                                                   discriminator.parameters()),
                                  lr = opt.lr, betas = (opt.b1, opt.b2))

In [20]:
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [21]:
# Static generator inputs for sampling
static_z = Variable(FloatTensor(np.zeros((opt.n_classes ** 2, opt.latent_dim))))
static_label = to_categorical(np.array([num for _ in range(opt.n_classes) for num in range(opt.n_classes)]),
                              num_columns = opt.n_classes
                             )
static_code = Variable(FloatTensor(np.zeros((opt.n_classes ** 2, opt.code_dim))))

## 4.4 Sample Image

In [22]:
def sample_image(n_row, batches_done):
    """
        Saves a grid of generated digits ranging from 0 to n_classes
    """
    # Static sample
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    static_sample = generator(z, static_label, static_code)
    save_image(static_sample.data, "images/static/%d.png" % batches_done, nrow = n_row, normalize = True)
    
    # Get varied c1 and c2
    zeros = np.zeros((n_row ** 2, 1))
    c_varied = np.repeat(np.linspace(-1, 1, n_row)[:, np.newaxis], n_row, 0)
    c1 = Variable(FloatTensor(np.concatenate((c_varied, zeros), -1)))
    c2 = Variable(FloatTensor(np.concatenate((zeros, c_varied), -1)))
    sample1 = generator(static_z, static_label, c1)
    sample2 = generator(static_z, static_label, c2)
    save_image(sample1.data, "images/varying_c1/%d.png" % batches_done, nrow = n_row, normalize = True)
    save_image(sample2.data, "images/varying_c2/%d.png" % batches_done, nrow = n_row, normalize = True)

## 4.5 Model Training

In [23]:
for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        batch_size = imgs.shape[0]
        
        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad = False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad = False)
        
        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = to_categorical(labels.numpy(), num_columns = opt.n_classes)
        
        ###################
        # Train Generator #
        ###################
        
        optimizer_G.zero_grad()
        
        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        label_input = to_categorical(np.random.randint(0, opt.n_classes, batch_size), num_columns = opt.n_classes)
        code_input = Variable(FloatTensor(np.random.uniform(-1, 1, (batch_size, opt.code_dim))))
        
        # Generate a batch of images
        gen_imgs = generator(z, label_input, code_input)
        
        # Loss measures generator's ability to fool the discriminator
        validity, _, _ = discriminator(gen_imgs)
        g_loss = adversarial_loss(validity, valid)
        
        g_loss.backward()
        optimizer_G.step()
        
        
        #######################
        # Train Discriminator #
        #######################
        
        optimizer_D.zero_grad()
        
        # Loss for real images
        real_pred, _, _ = discriminator(real_imgs)
        d_real_loss = adversarial_loss(real_pred, valid)
        
        # Loss for fake images
        fake_pred, _, _ = discriminator(gen_imgs.detach())
        d_fake_loss = adversarial_loss(fake_pred, fake)
        
        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2
        
        d_loss.backward()
        optimizer_D.step()
        
        
        ####################
        # Information Loss #
        ####################
        
        optimizer_info.zero_grad()
        
        # Sample labels
        sampled_labels = np.random.randint(0, opt.n_classes, batch_size)
        
        # Ground truth labels
        gt_labels = Variable(LongTensor(sampled_labels), requires_grad = False)
        
        # Sample noise, labels and code as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        label_input = to_categorical(sampled_labels, num_columns = opt.n_classes)
        code_input = Variable(FloatTensor(np.random.uniform(-1, 1, (batch_size, opt.code_dim))))
        
        gen_imgs = generator(z, label_input, code_input)
        _, pred_label, pred_code = discriminator(gen_imgs)
        
        info_loss = lambda_cat * categorical_loss(pred_label, gt_labels) + lambda_con * continuous_loss(pred_code, code_input)
            
        info_loss.backward()
        optimizer_info.step()
        
        ################
        # Log Progress #
        ################
        
        if i % 20 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [info loss: %f]"
                % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item(), info_loss.item())
            )
            
        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[Epoch 0/5] [Batch 0/938] [D loss: 0.505319] [G loss: 1.010333] [info loss: 2.336777]
[Epoch 0/5] [Batch 20/938] [D loss: 0.393967] [G loss: 0.746975] [info loss: 2.333571]
[Epoch 0/5] [Batch 40/938] [D loss: 0.212067] [G loss: 0.326726] [info loss: 2.328879]
[Epoch 0/5] [Batch 60/938] [D loss: 0.075085] [G loss: 0.653329] [info loss: 2.256758]
[Epoch 0/5] [Batch 80/938] [D loss: 0.099803] [G loss: 0.777411] [info loss: 2.034130]
[Epoch 0/5] [Batch 100/938] [D loss: 0.117710] [G loss: 0.928423] [info loss: 1.702694]
[Epoch 0/5] [Batch 120/938] [D loss: 0.134788] [G loss: 0.714284] [info loss: 1.617078]
[Epoch 0/5] [Batch 140/938] [D loss: 0.141017] [G loss: 0.601593] [info loss: 1.604203]
[Epoch 0/5] [Batch 160/938] [D loss: 0.204894] [G loss: 0.441721] [info loss: 1.566506]
[Epoch 0/5] [Batch 180/938] [D loss: 0.220423] [G loss: 0.470623] [info loss: 1.586051]
[Epoch 0/5] [Batch 200/938] [D loss: 0.193920] [G loss: 0.371933] [info loss: 1.549565]
[Epoch 0/5] [Batch 220/938] [D loss: 0